<a href="https://colab.research.google.com/github/nlaongtup/Habitat-sim/blob/main/Faster_RL_Training_Profiling_and_Optimization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Intro and motivation
>>* In this tutorial, we'll install, run, profile, and optimize an example Habitat RL train program, all in real time on this Colab machine.
>>* You can apply the lessons here to your own reinforcement-learning training.
>>* As you browse this Colab, I recommend you also follow along in the [YouTube video tutorial](https://youtu.be/I4MjX598ZYs). This live version of the Colab is slightly updated from the version recorded in the video.
* See all our Habitat tutorials in this series at [aihabitat.org/tutorial/2020](http://www.aihabitat.org/tutorial/2020).

## Why spend effort optimizing our training?
>>* Speed and efficiency are core values of the Habitat framework.
* High framerates let us scale our RL training to tens of millions of steps, as demonstrated in the [Habitat ICCV19 paper](https://arxiv.org/abs/1904.01201).
* High framerates also let us iterate on experiments faster, even at smaller scale.

## Our definition of profiling
>>* Measuring where our program spends time

## Why spend effort profiling our training program?
>>* We may not know what parts of our program are slow, or worse, we may have wrong assumptions here. The profile guides our optimization efforts.
* Later, as we start to optimize, our profiling framework will help us measure and verify speedup.

# Colab setup

>> We download and run a shell script to install Habitat on this machine, including some code specific to this tutorial. This will take a few minutes.






In [ ]:
!curl -L https://raw.githubusercontent.com/eundersander/habitat-api/eundersander/profiling_tutorial/colab_install_profiling.sh | bash -s

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  1846  100  1846    0     0   8061      0 --:--:-- --:--:-- --:--:--  8061
--2020-08-25 01:09:52--  https://repo.continuum.io/miniconda/Miniconda3-latest-Linux-x86_64.sh
Resolving repo.continuum.io (repo.continuum.io)... 104.18.200.79, 104.18.201.79, 2606:4700::6812:c84f, ...
Connecting to repo.continuum.io (repo.continuum.io)|104.18.200.79|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://repo.anaconda.com/miniconda/Miniconda3-latest-Linux-x86_64.sh [following]
--2020-08-25 01:09:52--  https://repo.anaconda.com/miniconda/Miniconda3-latest-Linux-x86_64.sh
Resolving repo.anaconda.com (repo.anaconda.com)... 104.16.131.3, 104.16.130.3, 2606:4700::6810:8303, ...
Connecting to repo.anaconda.com (repo.anaconda.com)|104.16.131.3|:443... connected.
HTTP request sent, awaiting response..

In [ ]:
%cd /content/habitat-api/
!git checkout `git log eundersander/profiling_tutorial --grep="add range_push/range_pop for major functions" --pretty=format:"%h"`

/content/habitat-api
Note: checking out 'eb2589d'.

You are in 'detached HEAD' state. You can look around, make experimental
changes and commit them, and you can discard any commits you make in this
state without impacting any branches by performing another checkout.

If you want to create a new branch to retain commits you create, you may
do so (now or later) by using -b with the checkout command again. Example:

  git checkout -b <new-branch-name>

HEAD is now at eb2589d add range_push/range_pop for major functions


# Our example train program
>>* PointGoal navigation using Proximal Policy Optimization (PPO) ([Schulman et al., 2017](https://arxiv.org/abs/1707.06347))
* A modified version of the [ppo_pointnav_example](https://github.com/facebookresearch/habitat-lab/blob/master/habitat_baselines/config/pointnav/ppo_pointnav_example.yaml) baseline in Habitat-Lab
* For more info on the PointGoal RL task, see our tutorial "Habitat-Lab for Navigation". [aihabitat.org/tutorial/2020](http://aihabitat.org/tutorial/2020/)
* I’ve modified the action space and reward function to better illustrate some profiling concepts.
* A main thread and 2 environment worker threads.

![PPO pseudo-code](https://drive.google.com/u/0/uc?id=1D7G2VdR6uN85-h7Ag8rgIOku7GVRucJx&export=download)

## Pseudocode
main thread, ppo_trainer.py:

```
def train():
  # create worker threads for N=2 environments
  construct_envs()
  for update in range(NUM_UPDATES):
    for timestep in range(T):
      # act (run policy) and step for N=2 environments;
      # env stepping is delegated to worker threads
      _collect_rollout_step()
    # compute surrogate loss, then optimize
    _update_agent()
```
environment worker thread, vector_env.py:
```
def _worker_env():
  # event loop
  while True:
    # wait to receive a command from the main thread
    command, data = recv()
    if command == STEP_COMMAND:
      observations, reward = env.step()
      # send results back to main thread
      send(observations, reward)
    elif command == RESET_COMMAND:
      # we'll reset periodically, immediately 
      # after the last step of an episode
      env.reset()
```


# Profiling and optimizing overview
>>Here's an overview of the process we'll use in this tutorial:
1. Create a short, representative “profiling config” for our train program.
2. Capture a profile with Py-spy, a sampling profiler.
3. Analyze the profile using Speedscope. Identify the major functions contributing to our runtime as well as candidates for optimization.
4. Annotate (mark) the major parts of our program in code.
5. Capture a second profile with higher timing precision, using another profiler, Nvidia Nsight Systems.
6. Try some optimizations.
7. Re-run Nsight Systems to evaluate the speedup.


# Create a profiling config
>>* A typical RL train run takes hours or days.
* Here, we want to enable a much faster iteration cycle of profiling and optimizing.
* So, we author a "profiling config", a modified version of the program that does a small, fixed amount of work.
* Prefer a fixed number of train steps rather than a fixed runtime duration.



In [ ]:
!head -10 habitat_baselines/config/pointnav/ppo_pointnav_profiling.yaml

BASE_TASK_CONFIG_PATH: "configs/tasks/pointnav.yaml"
TRAINER_NAME: "ppo"
ENV_NAME: "NavRLEnv"
VIDEO_OPTION: []
NUM_PROCESSES: 2
SENSORS: ["RGB_SENSOR", "DEPTH_SENSOR"]
NUM_UPDATES: 100
LOG_INTERVAL: 10
CHECKPOINT_INTERVAL: 1000



>>To keep this tutorial simple, we train from scratch and quit after just 100 updates. When you start to profile your own program, take care that your profiling config is ***representative*** of the full program:
* Training from scratch for 100 updates might not be as representative as starting from an intermediate checkpoint and training for 100 updates, because your agent probably behaves very poorly at the start of training.
* Some 3D scenes or training samples might not be representative in terms of visual complexity and rendering time.
* Training for too few steps may cause your profiling to miss infrequent events like logging, loading a new 3D scene, or checkpointing.

## Test the profiling config
>>* Let's run our program and make sure it completes without error. This will take a couple minutes.
* Note that I'm creating a little shell script here--that's just to make my command-line usage more readable and reusable in Colab.


In [ ]:
%%bash
(echo "python habitat_baselines/run.py --exp-config \
habitat_baselines/config/pointnav/ppo_pointnav_profiling.yaml \
--run-type train" > my_program.sh
chmod +x my_program.sh)

In [ ]:
!./my_program.sh

Neither `ifconfig` (`ifconfig -a`) nor `ip` (`ip address show`) commands are available, listing network interfaces is likely to fail
2020-08-25 01:12:45,897 config: BASE_TASK_CONFIG_PATH: configs/tasks/pointnav.yaml
CHECKPOINT_FOLDER: data/checkpoints
CHECKPOINT_INTERVAL: 1000
CMD_TRAILING_OPTS: []
ENV_NAME: NavRLEnv
EVAL:
  SPLIT: val
  USE_CKPT_CONFIG: True
EVAL_CKPT_PATH_DIR: data/checkpoints
FORCE_BLIND_POLICY: False
LOG_FILE: train.log
LOG_INTERVAL: 10
NUM_PROCESSES: 2
NUM_UPDATES: 100
ORBSLAM2:
  ANGLE_TH: 0.2617993877991494
  BETA: 100
  CAMERA_HEIGHT: 1.25
  DEPTH_DENORM: 10.0
  DIST_REACHED_TH: 0.15
  DIST_TO_STOP: 0.05
  D_OBSTACLE_MAX: 4.0
  D_OBSTACLE_MIN: 0.1
  H_OBSTACLE_MAX: 1.25
  H_OBSTACLE_MIN: 0.375
  MAP_CELL_SIZE: 0.1
  MAP_SIZE: 40
  MIN_PTS_IN_OBSTACLE: 320.0
  NEXT_WAYPOINT_TH: 0.5
  NUM_ACTIONS: 3
  PLANNER_MAX_STEPS: 500
  PREPROCESS_MAP: True
  SLAM_SETTINGS_PATH: habitat_baselines/slambased/data/mp3d3_small1k.yaml
  SLAM_VOCAB_PATH: habitat_baselines/slambas

# Py-spy
>>* py-spy is a sampling profiler for Python.
* While attached to our running program, it records our program's python call stack at regular intervals. We'll get an approximate measure of where time is spent.
* Learn more about py-spy here: [github.com/benfred/py-spy](https://github.com/benfred/py-spy)

## Capture a profile with py-spy
>>* We run py-spy from the command line, passing our program as a command-line argument.
* py-spy will run our program and output a profile (`pyspy_profile.speedscope`).

In [ ]:
%%bash
(pip install py-spy==0.4.0.dev1
echo "py-spy record --idle --function --native --subprocesses --rate 50 \
--output pyspy_profile.speedscope --format speedscope -- python \
habitat_baselines/run.py --exp-config \
habitat_baselines/config/pointnav/ppo_pointnav_profiling.yaml --run-type \
train" > my_pyspy_capture.sh
chmod +x my_pyspy_capture.sh)

In [ ]:
!./my_pyspy_capture.sh

py-spy> Sampling process 50 times a second. Press Control-C to exit.

Neither `ifconfig` (`ifconfig -a`) nor `ip` (`ip address show`) commands are available, listing network interfaces is likely to fail
2020-08-25 01:14:04,141 config: BASE_TASK_CONFIG_PATH: configs/tasks/pointnav.yaml
CHECKPOINT_FOLDER: data/checkpoints
CHECKPOINT_INTERVAL: 1000
CMD_TRAILING_OPTS: []
ENV_NAME: NavRLEnv
EVAL:
  SPLIT: val
  USE_CKPT_CONFIG: True
EVAL_CKPT_PATH_DIR: data/checkpoints
FORCE_BLIND_POLICY: False
LOG_FILE: train.log
LOG_INTERVAL: 10
NUM_PROCESSES: 2
NUM_UPDATES: 100
ORBSLAM2:
  ANGLE_TH: 0.2617993877991494
  BETA: 100
  CAMERA_HEIGHT: 1.25
  DEPTH_DENORM: 10.0
  DIST_REACHED_TH: 0.15
  DIST_TO_STOP: 0.05
  D_OBSTACLE_MAX: 4.0
  D_OBSTACLE_MIN: 0.1
  H_OBSTACLE_MAX: 1.25
  H_OBSTACLE_MIN: 0.375
  MAP_CELL_SIZE: 0.1
  MAP_SIZE: 40
  MIN_PTS_IN_OBSTACLE: 320.0
  NEXT_WAYPOINT_TH: 0.5
  NUM_ACTIONS: 3
  PLANNER_MAX_STEPS: 500
  PREPROCESS_MAP: True
  SLAM_SETTINGS_PATH: habitat_baselines/slambase

## Analyze the py-spy profile
>>* Run the code block below to download the py-spy profile to your local machine.
* Browse to [www.speedscope.app](http://www.speedscope.app) and drag-and-drop your file.
* Follow along in the video tutorial, starting at [3:25](https://youtu.be/I4MjX598ZYs?t=205), as we explore the profile in the speedscope interactive viewer.
* Learn more about speedscope here: [github.com/jlfwong/speedscope](https://github.com/jlfwong/speedscope)

In [ ]:
from google.colab import files
files.download("pyspy_profile.speedscope")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

## Py-spy takeaways
>>In the video segment, what did we accomplish with py-spy and speedscope?
* We identified the major python functions contributing to our runtime, across multiple threads.
* We sanity-checked that these major functions match our expectations from earlier pseudocode.
* We identified two unexpected expensive functions: `update_metrics` (our modified reward function, which calls `get_walkability_score`) and `before_step` (which calls `clip_grad_norm`). We'll address them shortly.

>>What are some limitations of py-spy?
* Because it's a sampling profiler, py-spy doesn't produce high-precision timings of function call durations. This will be essential as we pursue fine-grain optimizations.
* It doesn't offer a accurate multithreaded trace (time-ordered view) of function calls. If you tried the Time Order view in speedscope, you may have noticed missing function calls and other inconsistencies. This is again because py-spy is sample-based. An accurate multithreaded trace will be useful as we look for opportunities for better parallelization between threads. 

# Nsight Systems
>>* As mentioned earlier, sampling profilers like py-spy have limitations.
* Next, we're going to switch to a profiler from Nvidia that supports manual annotation, and we're going to annotate (mark) the major parts of our program identified in the Py-spy profile.
* Nsight Systems has additional capabilities related to CUDA and graphics APIs like OpenGL, but for now, we'll just use it to capture annotations.
* Learn more about Nsight Systems here: [developer.nvidia.com/nsight-systems](https://developer.nvidia.com/nsight-systems)

## Annotate our program
>>* Our annotation API comes from Nvidia NVTX, and we'll wrap this API with `profiling_utils.range_push` and `range_pop`.
* Find the latest version of `profiling_utils` in the Habitat-Sim github repo: [`profiling_utils.py`](https://github.com/facebookresearch/habitat-sim/blob/master/habitat_sim/utils/profiling_utils.py).
* The Nsight profiler will record the CPU time spent in these ranges.
* Use `range_push` to mark the start of a section of code and always use a corresponding `range_pop` to mark the end.

In [ ]:
!cat habitat/utils/profiling_utils.py

import torch

def range_push(msg):
    torch.cuda.nvtx.range_push(msg)

def range_pop():
    torch.cuda.nvtx.range_pop()

>>To illustrate how to use these, here's pseudocode for marking our top-level `train` function and the train loop body.

```
from habitat.utils import profiling_utils

def train():
  profiling_utils.range_push("train")
  construct_envs()
  for update in range(NUM_UPDATES):
    profiling_utils.range_push("train loop body")
    for timestep in range(T):
      _collect_rollout_step()
    _update_agent()
    profiling_utils.range_pop()  # train loop body
  profiling_utils.range_pop()  # train
```

>>We're going to mark all the major parts of our program identified earlier, about 15 ranges in total:

In [ ]:
!git log -p -U0 --grep="Add ranges for entire program and train loop body"

commit a40f46938aa6d7006b9ebd4247d847070af5b2b2
Author: Eric Undersander <eundersander@gmail.com>
Date:   Mon Jul 27 22:57:41 2020 -0700

    Add ranges for entire program and train loop body

diff --git a/habitat_baselines/rl/ppo/ppo_trainer.py b/habitat_baselines/rl/ppo/ppo_trainer.py
index 4b2c4f0..71a9aaf 100644
--- a/habitat_baselines/rl/ppo/ppo_trainer.py
+++ b/habitat_baselines/rl/ppo/ppo_trainer.py
@@ -31,0 +32 @@ from habitat_baselines.rl.ppo import PPO, PointNavBaselinePolicy
+from habitat.utils import profiling_utils
@@ -271,0 +273 @@ class PPOTrainer(BaseRLTrainer):
+        profiling_utils.range_push("train")
@@ -336,0 +339 @@ class PPOTrainer(BaseRLTrainer):
+                profiling_utils.range_push("train loop body")
@@ -431 +434 @@ class PPOTrainer(BaseRLTrainer):
-
+                profiling_utils.range_pop()  # train loop body
@@ -433,0 +437,2 @@ class PPOTrainer(BaseRLTrainer):
+        profiling_utils.range_pop()  # train
+


In [ ]:
!git log eundersander/profiling_tutorial -p -U0 --grep="add range_push/range_pop for major functions"

commit eb2589de881066186e61bd3bf1e5ccf665ff631f (HEAD)
Author: Eric Undersander <eundersander@gmail.com>
Date:   Wed Aug 5 22:52:42 2020 -0700

    add range_push/range_pop for major functions

diff --git a/habitat/core/vector_env.py b/habitat/core/vector_env.py
index 0a01bbd..b800ce2 100644
--- a/habitat/core/vector_env.py
+++ b/habitat/core/vector_env.py
@@ -32,0 +33 @@ from habitat.core.utils import tile_images
+from habitat.utils import profiling_utils
@@ -178,0 +180 @@ class VectorEnv:
+        profiling_utils.range_push("_worker_env")
@@ -200,0 +203 @@ class VectorEnv:
+                        profiling_utils.range_push("_worker_env send")
@@ -201,0 +205 @@ class VectorEnv:
+                        profiling_utils.range_pop()  # _worker_env send
@@ -248,0 +253 @@ class VectorEnv:
+                profiling_utils.range_push("_worker_env recv")
@@ -249,0 +255 @@ class VectorEnv:
+                profiling_utils.range_pop()  # _worker_env recv
@@ -256,0 +263 @@ class VectorEnv:
+   

## Capture a profile with Nsight
>>* As with py-spy, Nsight Systems can be invoked from the command line (`nsys`) to capture a profile.
* We specify `--trace=nvtx`. Nsight will record every entry and exit time for all our annotated ranges.
* Nsight will output a profile in a proprietary `qdrep` format. With the `--export=sqlite` argument, it will also output the same information as an SQLite database file.

In [ ]:
%%bash
(apt-get install -y nvidia-nsight
echo "/usr/local/bin/nsys profile --sample=none --trace=nvtx \
--trace-fork-before-exec=true --output=\$1 --export=sqlite python \
habitat_baselines/run.py --exp-config \
habitat_baselines/config/pointnav/ppo_pointnav_profiling.yaml \
--run-type train" > my_nsight_capture.sh
chmod +x my_nsight_capture.sh)

Reading package lists...
Building dependency tree...
Reading state information...
The following package was automatically installed and is no longer required:
  libnvidia-common-440
Use 'apt autoremove' to remove it.
The following additional packages will be installed:
  fonts-dejavu-core fonts-dejavu-extra libatk-wrapper-java
  libatk-wrapper-java-jni libxxf86dga1 nvidia-cuda-doc openjdk-8-jre
  openjdk-8-jre-headless x11-utils
Suggested packages:
  icedtea-8-plugin libnss-mdns fonts-ipafont-gothic fonts-ipafont-mincho
  fonts-wqy-microhei fonts-wqy-zenhei fonts-indic mesa-utils
The following NEW packages will be installed:
  fonts-dejavu-core fonts-dejavu-extra libatk-wrapper-java
  libatk-wrapper-java-jni libxxf86dga1 nvidia-cuda-doc nvidia-nsight
  openjdk-8-jre openjdk-8-jre-headless x11-utils
0 upgraded, 10 newly installed, 0 to remove and 35 not upgraded.
Need to get 282 MB of archives.
After this operation, 510 MB of additional disk space will be used.
Get:1 http://archive.ubun

E: Failed to fetch http://security.ubuntu.com/ubuntu/pool/universe/o/openjdk-8/openjdk-8-jre-headless_8u252-b09-1~18.04_amd64.deb  404  Not Found [IP: 91.189.88.142 80]
E: Failed to fetch http://security.ubuntu.com/ubuntu/pool/universe/o/openjdk-8/openjdk-8-jre_8u252-b09-1~18.04_amd64.deb  404  Not Found [IP: 91.189.88.142 80]
E: Unable to fetch some archives, maybe run apt-get update or try with --fix-missing?


In [ ]:
!./my_nsight_capture.sh 0_baseline

Neither `ifconfig` (`ifconfig -a`) nor `ip` (`ip address show`) commands are available, listing network interfaces is likely to fail
2020-08-25 01:15:50,497 config: BASE_TASK_CONFIG_PATH: configs/tasks/pointnav.yaml
CHECKPOINT_FOLDER: data/checkpoints
CHECKPOINT_INTERVAL: 1000
CMD_TRAILING_OPTS: []
ENV_NAME: NavRLEnv
EVAL:
  SPLIT: val
  USE_CKPT_CONFIG: True
EVAL_CKPT_PATH_DIR: data/checkpoints
FORCE_BLIND_POLICY: False
LOG_FILE: train.log
LOG_INTERVAL: 10
NUM_PROCESSES: 2
NUM_UPDATES: 100
ORBSLAM2:
  ANGLE_TH: 0.2617993877991494
  BETA: 100
  CAMERA_HEIGHT: 1.25
  DEPTH_DENORM: 10.0
  DIST_REACHED_TH: 0.15
  DIST_TO_STOP: 0.05
  D_OBSTACLE_MAX: 4.0
  D_OBSTACLE_MIN: 0.1
  H_OBSTACLE_MAX: 1.25
  H_OBSTACLE_MIN: 0.375
  MAP_CELL_SIZE: 0.1
  MAP_SIZE: 40
  MIN_PTS_IN_OBSTACLE: 320.0
  NEXT_WAYPOINT_TH: 0.5
  NUM_ACTIONS: 3
  PLANNER_MAX_STEPS: 500
  PREPROCESS_MAP: True
  SLAM_SETTINGS_PATH: habitat_baselines/slambased/data/mp3d3_small1k.yaml
  SLAM_VOCAB_PATH: habitat_baselines/slambas

## Analyze summary timings
>>* Habitat provides a `compare_profiles` script to print summary timings from Nsight's exported SQLite database file.
* Find the latest version of `compare_profiles` in the Habitat-Sim github repo: [`compare_profiles.py`](https://github.com/facebookresearch/habitat-sim/blob/master/habitat_sim/utils/compare_profiles.py).
* We see total time spent in each of our annotated ranges.
* Inclusive time represents the total time spent in a range, including any nested (child) ranges.
* Exclusive time is time spent "only" in that range, not any child ranges.
* The inclusive time for `train` corresponds to our main thread's total runtime.
* Our 2 environment workers accumulate time into the same named ranges, like `_worker_env` and `habitat_simulator.py step`, so those totals appear doubled.


In [ ]:
!python habitat/utils/compare_profiles.py

                                       0_baseline.sqlite           
event name                                incl (ms)     excl (ms)  
_worker_env                                 121,394         8,543  
train                                        63,842        10,409  
_worker_env recv                             58,752        58,752  
train loop body                              53,431           836  
_collect_rollout_step                        42,976         2,301  
wait_step                                    31,669        31,669  
_worker_env send                             24,724        24,724  
habitat_simulator.py step                    18,904        18,904  
nav.py update_metric                         10,472        10,472  
_update_agent                                 9,619         1,674  
act (run policy)                              5,399         5,399  
evaluate_actions (for surrogate loss)         3,798         3,798  
batch_obs                                     3,

>>Our profiling framework is in place, so we're now ready to start optimizing.

# Optimize!

>>* Let's first address `update_metric` and `get_walkability_score`.
* We see that `get_walkability_score` is querying `is_navigable_path` in a loop, using randomly-generated points. This appears to be some kind of sampling.
* With the runtime cost now known, I can make a smarter choice about the tradeoff between accuracy and runtime cost.
* I choose to decrease the number of samples from 256 to 64.

In [ ]:
!grep habitat/tasks/nav/nav.py -e "def get_walkability_score" -A14

    def get_walkability_score(self, position):
        num_reachable_points = 0
        max_reachable_points = 256
        query_range = 10
        # Randomly sample nearby points. Compute the fraction to which we can navigate.
        # Call this fraction our walkability score. This is a contrived, CPU-intensive
        # operation for our profiling tutorial.
        for i in range(max_reachable_points):
            query_pos = position + \
                [random.randrange(-query_range, query_range), 0, random.randrange(-query_range, query_range)]
            if self._sim.is_navigable_path(position, query_pos):
                num_reachable_points += 1

        score = num_reachable_points / max_reachable_points
        return score


In [ ]:
!git log eundersander/profiling_tutorial -p -U0 --grep="change max_reachable_points from 256 to 64"
!git checkout `git log eundersander/profiling_tutorial --grep="change max_reachable_points from 256 to 64" --pretty=format:"%h"`

commit f5c2d5e9648de8ec697c909feb0eae2fb6e3c214
Author: Eric Undersander <eundersander@gmail.com>
Date:   Wed Aug 5 23:59:22 2020 -0700

    change max_reachable_points from 256 to 64

diff --git a/habitat/tasks/nav/nav.py b/habitat/tasks/nav/nav.py
index b936dfd..e962962 100644
--- a/habitat/tasks/nav/nav.py
+++ b/habitat/tasks/nav/nav.py
@@ -985 +985 @@ class DistanceToGoal(Measure):
-        max_reachable_points = 256
+        max_reachable_points = 64
Previous HEAD position was eb2589d add range_push/range_pop for major functions
HEAD is now at f5c2d5e change max_reachable_points from 256 to 64


## Evaluate first optimization
>>* Profiling and optimizing is an iterative process.
* Let's evaluate our first optimization and see if we get a speedup.
* We evaluate by simply re-running our Nsight capture script. This will output a new profile alongside the old one.

In [ ]:
!./my_nsight_capture.sh 1_walkability

Neither `ifconfig` (`ifconfig -a`) nor `ip` (`ip address show`) commands are available, listing network interfaces is likely to fail
2020-08-25 01:17:00,189 config: BASE_TASK_CONFIG_PATH: configs/tasks/pointnav.yaml
CHECKPOINT_FOLDER: data/checkpoints
CHECKPOINT_INTERVAL: 1000
CMD_TRAILING_OPTS: []
ENV_NAME: NavRLEnv
EVAL:
  SPLIT: val
  USE_CKPT_CONFIG: True
EVAL_CKPT_PATH_DIR: data/checkpoints
FORCE_BLIND_POLICY: False
LOG_FILE: train.log
LOG_INTERVAL: 10
NUM_PROCESSES: 2
NUM_UPDATES: 100
ORBSLAM2:
  ANGLE_TH: 0.2617993877991494
  BETA: 100
  CAMERA_HEIGHT: 1.25
  DEPTH_DENORM: 10.0
  DIST_REACHED_TH: 0.15
  DIST_TO_STOP: 0.05
  D_OBSTACLE_MAX: 4.0
  D_OBSTACLE_MIN: 0.1
  H_OBSTACLE_MAX: 1.25
  H_OBSTACLE_MIN: 0.375
  MAP_CELL_SIZE: 0.1
  MAP_SIZE: 40
  MIN_PTS_IN_OBSTACLE: 320.0
  NEXT_WAYPOINT_TH: 0.5
  NUM_ACTIONS: 3
  PLANNER_MAX_STEPS: 500
  PREPROCESS_MAP: True
  SLAM_SETTINGS_PATH: habitat_baselines/slambased/data/mp3d3_small1k.yaml
  SLAM_VOCAB_PATH: habitat_baselines/slambas

>>* Here, we used `--relative` to show the time delta between `0_baseline` and `1_walkability`.
* On the environment worker threads, `nav.py update_metric` shows a big improvement corresponding to our reduction from 256 to 64 samples.
* On the main thread, `wait_step` and (most importantly) `train` also show a corresponding improvement. Success! Our training is faster!


In [ ]:
!python habitat/utils/compare_profiles.py --relative

                                       0_baseline.sqlite           1_walkability.sqlite        
event name                                incl (ms)     excl (ms)  
_worker_env                                 121,394         8,543       -14,364          -151  
train                                        63,842        10,409        -7,378          -471  
_worker_env recv                             58,752        58,752        -5,348        -5,348  
train loop body                              53,431           836        -6,907          -495  
_collect_rollout_step                        42,976         2,301        -6,403           -77  
wait_step                                    31,669        31,669        -5,310        -5,310  
_worker_env send                             24,724        24,724        -2,221        -2,221  
habitat_simulator.py step                    18,904        18,904          -476          -476  
nav.py update_metric                         10,472        10,472   

## Optimize more!
>>* Let's also address `before_step` and `clip_grad_norm_`.
* As we discussed earlier, it's suspicious that `clip_grad_norm_` should be so expensive.
* Some googling suggests a few other PyTorch users have encountered performance problems with this function.
* `clip_grad_norm_` clips the length (norm) of the entire gradient vector, while `clip_grad_value_` does an element-wise clamp. 
* As a performance experiment, let's try `clip_grad_value_`. In the long term, this different type of gradient-clipping may or may not be feasible for our training. Regardless, it will be valuable to know if it offers a speedup.

In [ ]:
!git log eundersander/profiling_tutorial -p -U0 --grep="try clip_grad_value_ instead of clip_grad_norm_, as a perf test"
!git checkout `git log eundersander/profiling_tutorial --grep="try clip_grad_value_ instead of clip_grad_norm_, as a perf test" --pretty=format:"%h"`

commit 81c70f3fd516ef0414391b85c46a892c515e9cdf
Author: Eric Undersander <eundersander@gmail.com>
Date:   Thu Aug 6 00:00:33 2020 -0700

    try clip_grad_value_ instead of clip_grad_norm_, as a perf test

diff --git a/habitat_baselines/rl/ppo/ppo.py b/habitat_baselines/rl/ppo/ppo.py
index a1db76b..30d953b 100644
--- a/habitat_baselines/rl/ppo/ppo.py
+++ b/habitat_baselines/rl/ppo/ppo.py
@@ -169,3 +169 @@ class PPO(nn.Module):
-        nn.utils.clip_grad_norm_(
-            self.actor_critic.parameters(), self.max_grad_norm
-        )
+        nn.utils.clip_grad_value_(self.actor_critic.parameters(), 0.1)
Previous HEAD position was f5c2d5e change max_reachable_points from 256 to 64
HEAD is now at 81c70f3 try clip_grad_value_ instead of clip_grad_norm_, as a perf test


## Evaluate second optimization

In [ ]:
!./my_nsight_capture.sh 2_clipgrad

Neither `ifconfig` (`ifconfig -a`) nor `ip` (`ip address show`) commands are available, listing network interfaces is likely to fail
2020-08-25 01:18:01,052 config: BASE_TASK_CONFIG_PATH: configs/tasks/pointnav.yaml
CHECKPOINT_FOLDER: data/checkpoints
CHECKPOINT_INTERVAL: 1000
CMD_TRAILING_OPTS: []
ENV_NAME: NavRLEnv
EVAL:
  SPLIT: val
  USE_CKPT_CONFIG: True
EVAL_CKPT_PATH_DIR: data/checkpoints
FORCE_BLIND_POLICY: False
LOG_FILE: train.log
LOG_INTERVAL: 10
NUM_PROCESSES: 2
NUM_UPDATES: 100
ORBSLAM2:
  ANGLE_TH: 0.2617993877991494
  BETA: 100
  CAMERA_HEIGHT: 1.25
  DEPTH_DENORM: 10.0
  DIST_REACHED_TH: 0.15
  DIST_TO_STOP: 0.05
  D_OBSTACLE_MAX: 4.0
  D_OBSTACLE_MIN: 0.1
  H_OBSTACLE_MAX: 1.25
  H_OBSTACLE_MIN: 0.375
  MAP_CELL_SIZE: 0.1
  MAP_SIZE: 40
  MIN_PTS_IN_OBSTACLE: 320.0
  NEXT_WAYPOINT_TH: 0.5
  NUM_ACTIONS: 3
  PLANNER_MAX_STEPS: 500
  PREPROCESS_MAP: True
  SLAM_SETTINGS_PATH: habitat_baselines/slambased/data/mp3d3_small1k.yaml
  SLAM_VOCAB_PATH: habitat_baselines/slambas

>>* We compare `2_clipgrad` against the previous profile, `1_walkability`.














In [ ]:
!rm -f 0_baseline.sqlite
!python habitat/utils/compare_profiles.py --relative

                                       1_walkability.sqlite        2_clipgrad.sqlite           
event name                                incl (ms)     excl (ms)  
_worker_env                                 107,031         8,392          +612           -77  
train                                        56,463         9,939          +385          +105  
_worker_env recv                             53,404        53,404          +283          +283  
train loop body                              46,524           341          +280          +372  
_collect_rollout_step                        36,573         2,224          +396           -29  
wait_step                                    26,359        26,359          -258          -258  
_worker_env send                             22,503        22,503        +1,329        +1,329  
habitat_simulator.py step                    18,429        18,429          -289          -289  
_update_agent                                 9,610         1,688   

>>* `before_step` does indeed show a big improvement.
* However, there's no significant improvement to total program runtime, represented by `train`'s inclusive time.
* It may be that asynchronous GPU work is causing misleading CPU timing for `before_step`. We'll look at this more closely in the next section.
* Note that timing is sensitive to noise, such as randomness in your program or activity by other users on a shared machine. You can mitigate noise by capturing multiple profiles or increasing the number of train steps in your profiling config.
* Overall, it appears our switch to `clip_grad_value_` yields little to no train speedup.


# Analyze with Nsight GUI
>>* So far, we've used Nsight Systems's command-line tool (`nsys`) to capture our NVTX annotations, and we've used Habitat's `compare_profiles` script to view summary timings.
* In this section, we'll also capture our program's PyTorch CUDA usage, and we'll use Nsight's GUI tool to view a multithreaded trace of our program.
* To follow along here, you should install and run the Nsight Systems GUI tool on your Linux, Mac, or Windows desktop. [developer.nvidia.com/nsight-systems](https://developer.nvidia.com/nsight-systems)
* We'll investigate GPU usage and why our earlier change to `before_step` failed to improve total program runtime.
* We'll identify another optimization candidate related to multithreading.

In [ ]:
%%bash
(echo "/usr/local/bin/nsys profile --sample=none --trace=cuda,nvtx \
--trace-fork-before-exec=true --output=\$1 python \
habitat_baselines/run.py --exp-config \
habitat_baselines/config/pointnav/ppo_pointnav_profiling.yaml \
--run-type train" > my_nsight_capture_for_gui.sh
chmod +x ./my_nsight_capture_for_gui.sh)

>>* We revert our `clip_grad_value_` experiment but keep our first optimization related to `get_walkability_score`.
* We capture a CUDA-traced profile.

In [ ]:

!git checkout `git log eundersander/profiling_tutorial --grep="change max_reachable_points from 256 to 64" --pretty=format:"%h"`
!./my_nsight_capture_for_gui.sh 3_walkability

Previous HEAD position was 81c70f3 try clip_grad_value_ instead of clip_grad_norm_, as a perf test
HEAD is now at f5c2d5e change max_reachable_points from 256 to 64
Neither `ifconfig` (`ifconfig -a`) nor `ip` (`ip address show`) commands are available, listing network interfaces is likely to fail
2020-08-25 01:19:03,035 config: BASE_TASK_CONFIG_PATH: configs/tasks/pointnav.yaml
CHECKPOINT_FOLDER: data/checkpoints
CHECKPOINT_INTERVAL: 1000
CMD_TRAILING_OPTS: []
ENV_NAME: NavRLEnv
EVAL:
  SPLIT: val
  USE_CKPT_CONFIG: True
EVAL_CKPT_PATH_DIR: data/checkpoints
FORCE_BLIND_POLICY: False
LOG_FILE: train.log
LOG_INTERVAL: 10
NUM_PROCESSES: 2
NUM_UPDATES: 100
ORBSLAM2:
  ANGLE_TH: 0.2617993877991494
  BETA: 100
  CAMERA_HEIGHT: 1.25
  DEPTH_DENORM: 10.0
  DIST_REACHED_TH: 0.15
  DIST_TO_STOP: 0.05
  D_OBSTACLE_MAX: 4.0
  D_OBSTACLE_MIN: 0.1
  H_OBSTACLE_MAX: 1.25
  H_OBSTACLE_MIN: 0.375
  MAP_CELL_SIZE: 0.1
  MAP_SIZE: 40
  MIN_PTS_IN_OBSTACLE: 320.0
  NEXT_WAYPOINT_TH: 0.5
  NUM_ACTIONS: 3
 

>>* Download and open the `qdrep` profile in the Nsight Systems GUI tool on your desktop.
* Follow along in the video tutorial, starting at [13:10](https://youtu.be/I4MjX598ZYs?t=784), as we explore this profile.

In [ ]:
files.download("3_walkability.qdrep")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

>>What did we we learn in the video segment using the Nsight GUI?
* We looked at CUDA execution on the GPU for `_update_agent`. The GPU runtime for `clip_grad_norm` is small, which helps explain the results of our earlier experiment with `clip_grad_value_`.
* By looking at all threads together in the multithreaded trace view, we were able to spot another possible optimization related to `update_metric` and better parallelization.



# Review

>>Let's review the process we used in this tutorial:
1. We modified our train program to run for just 100 updates.
2. We used py-spy and speedscope to identify the major functions contributing to our runtime. We also identified two candidates for optimization.
3. We annotated the major parts of our program in code for use with Nsight Systems.
4. We tried some optimizations and evaluated their speedup using Nsight.
5. We used Nsight's multithreaded trace view to spot another optimization candidate related to parallelism. 


# Next steps



## Use this Colab to experiment

>>* You can directly edit files using the Files browser on the left.
* Try changing the experiment parameters in `habitat_baselines\config\pointnav\ppo_pointnav_profiling.yaml` or the vision model in `habitat_baselines\rl\models\simple_cnn.py`.
* Re-run profiling to check the resulting speedup/slowdown.
* To prototype our last optimization candidate (moving `update_metric` after `send`), run the cell below to get the change.
 * In my tests, I found it yielded a large speedup on my 6-core dev machine but negligible speedup on single-core Colab machines.

In [ ]:
!git log eundersander/profiling_tutorial -p -U3 --grep="run nav.py update_metric after send(), as a perf test"
!git checkout `git log eundersander/profiling_tutorial --grep="run nav.py update_metric after send(), as a perf test" --pretty=format:"%h"`

commit d64b8cb75e56ef557bfc7a131068431f20e54ec9 (origin/eundersander/profiling_tutorial, eundersander/profiling_tutorial)
Author: Eric Undersander <eundersander@gmail.com>
Date:   Wed Aug 5 22:46:14 2020 -0700

    run nav.py update_metric after send(), as a perf test

diff --git a/habitat/core/env.py b/habitat/core/env.py
index 6cc395a..dc62b67 100644
--- a/habitat/core/env.py
+++ b/habitat/core/env.py
@@ -263,14 +263,23 @@ class Env:
             action=action, episode=self.current_episode
         )
 
-        self._task.measurements.update_measures(
-            episode=self.current_episode, action=action, task=self.task
-        )
+        if False:
+            self._task.measurements.update_measures(
+                episode=self.current_episode, action=action, task=self.task
+            )
 
         self._update_step_stats()
 
         return observations
 
+    def post_step(self, action):
+        if True:
+            self._task.measurements.update_measures(
+              

## Explore Nsight Systems
>>* The tool has more capture features not covered in this tutorial: OpenGL and other graphics APIs, CPU utilization, sampling-based C++ callstacks, etc.
* On Linux, the tool works better with newer Nvidia drivers, e.g. `440`. I encountered some problems on machines with `418` (this includes Colab machines as of August 2020).
* If you're profiling a desktop machine, I recommend using the GUI tool to capture (instead of the `nsys` command-line tool). In the 2020.3 release, the GUI version has more features and options.

## Habitat Profiling Tools
>>* We're planning to integrate more profiling tools into Habitat-Sim and Habitat-Lab, so stay tuned:
 * [aihabitat.org](http://aihabitat.org/)
 * [github.com/facebookresearch/habitat-sim](https://github.com/facebookresearch/habitat-sim/)
 * [github.com/facebookresearch/habitat-lab](https://github.com/facebookresearch/habitat-lab/)
* If you have questions or suggestions for me or the Habitat team, reach out via Github or our website